In [ ]:
import os
import sys
from pathlib import Path
    
import copy

import numpy as np
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt

sns.set_context("paper", font_scale=2)
sns.set_style("whitegrid")

In [ ]:
BASE_PATH = Path("path/to/the/folder/hephaestus/is/located")

BASE_EXPERIMENT_DIRECTORY = Path("path/to/where/model_storage/models/is/located")
MODELS_RESULTS = os.path.join("model_storage", "models")

SAVE_PATH = BASE_PATH / "plots" / "compare_models"
SAVE_PATH.mkdir(exist_ok=True, parents=True)

In [ ]:
os.chdir(BASE_PATH)
sys.path.insert(0, BASE_PATH)

import hephaestus.utils.general_utils as hutils

In [ ]:
experiment_map = {
    "d_gcn": ["TALOS_20240110-163351", "TAUROI_KHALKEOI_20231231-004707"],
    "d_gat": ["TALOS_20240113-155714", "TAUROI_KHALKEOI_20240101-170302"],
    "d_sage": ["TALOS_20240119-150636", "TAUROI_KHALKEOI_20240108-205055"],
    "d_gin": ["TALOS_20240121-203233", "TAUROI_KHALKEOI_20240106-193249"],
    "nd_gin": ["TALOS_20240129-182636", "TALOS_20240201-180351"],
    "nd_gat": ["TALOS_20240203-234629", "TALOS_20240209-190133"],
    "nd_gcn": ["TALOS_20240212-124657"],
    "nd_sage": ["TALOS_20240216-185022"],
}


In [ ]:
experiment_map_2 = {}
for k in experiment_map.keys():
    for v in experiment_map[k]:
        experiment_map_2[v] = k

In [ ]:
experiment_map_2

In [ ]:
experiment_stats = {
    "d_gcn": 0,
    "d_gat": 0,
    "d_sage": 0,
    "d_gin": 0,
    "nd_gin": 0,
    "nd_gat": 0,
    "nd_gcn": 0,
    "nd_sage": 0,
}

experiment_progress = {
    "d_gcn": [],
    "d_gat": [],
    "d_sage": [],
    "d_gin": [],
    "nd_gin": [],
    "nd_gat": [],
    "nd_gcn": [],
    "nd_sage": [], 
}

for experiment in os.listdir(BASE_EXPERIMENT_DIRECTORY/MODELS_RESULTS):
    trial_cnt = 0
    for trial in os.listdir(BASE_EXPERIMENT_DIRECTORY/MODELS_RESULTS/experiment):
        if "TorchTrainer" not in trial:
            continue
        
        try:
            df = pd.read_csv(BASE_EXPERIMENT_DIRECTORY/MODELS_RESULTS/experiment/trial/"progress.csv")
            experiment_progress[experiment_map_2[experiment]].append(df)
            trial_cnt += 1
        except FileNotFoundError:
            print(experiment_map_2[experiment])
            print(experiment, trial, " is incomplete")
        except KeyError:
            print(experiment, "does not exist in the given dict")
            break

    try:
        experiment_stats[experiment_map_2[experiment]] += trial_cnt
    except KeyError:
        pass

In [ ]:
experiment_dfs = {}

value_vars_d = [
            "loss",
            "train_loss",
            "med_abs_error",
            "max_abs_err_pattern",
            "mean_worse_abs_errs_graph",

        ]
value_vars_nd = copy.deepcopy([value_vars_d])
value_vars_nd.append(['q11', 'q12', 'q13', 'q14', 'q15',
       'q21', 'q22', 'q23', 'q24', 'q25', 'q31', 'q32', 'q33', 'q34', 'q35',
       'q41', 'q42', 'q43', 'q44', 'q45', 'q51', 'q52', 'q53', 'q54', 'q55',
       'q61', 'q62', 'q63', 'q64', 'q65', 'q71', 'q72', 'q73', 'q74', 'q75',
       'q81', 'q82', 'q83', 'q84', 'q85'])
value_vars_nd = hutils.flatten_nested_list(value_vars_nd, sort=False)


for k in experiment_progress.keys():
    experiment_dfs[k] = pd.concat(experiment_progress[k])
    if "nd_" in k:
        vals = value_vars_nd
    else:
        vals = value_vars_d
        
    experiment_dfs[k + "_melt"] = pd.melt(
        experiment_dfs[k].drop(
            ["max_abs_err_pattern_idx", "mean_worse_abs_errs_graph_idx"], axis=1
        ),
        id_vars=[
            "timestamp",
            "checkpoint_dir_name",
            "should_checkpoint",
            "done",
            "training_iteration",
            "trial_id",
            "date",
            "time_this_iter_s",
            "time_total_s",
            "pid",
            "hostname",
            "node_ip",
            "time_since_restore",
            "iterations_since_restore",
        ],
        value_vars=vals,
        value_name="Score",
        var_name="Metric",
    )

In [ ]:
_t = []
for k in [k for k in experiment_dfs.keys() if "_melt" in k and "nd_" not in k]:
    df = experiment_dfs[k].copy(deep=True)
    df["GNN"] = k.split("_")[1]
    _t.append(df)
d_all_dfs = pd.concat(_t)

_t = []
for k in [k for k in experiment_dfs.keys() if "_melt" in k and "nd_" in k]:
    df = experiment_dfs[k].copy(deep=True)
    df["GNN"] = k.split("_")[1]
    _t.append(df)
nd_all_dfs = pd.concat(_t)
del _t

In [ ]:
sns.set_context("paper", font_scale=2.5)
sns.set_style("whitegrid")

In [ ]:
p = [
    "#000000",
    "#E69F00",
    "#56B4E9",
    "#009E73",
    "#FB6467FF",
    "#808282",
    "#F0E442",
    "#440154FF",
    "#0072B2",
    "#D55E00",
    "#CC79A7",
    "#C2CD23",
    "#918BC3",
    "#FFFFFF",
]

mpnn_pal = {"gcn": "#E69F00", "gin": "#56B4E9", "sage": "#FB6467FF", "gat": "#009E73"}

sns.color_palette(p)

In [ ]:
g = sns.FacetGrid(
    d_all_dfs,
    sharex=True,
    sharey=False,
    col="Metric",
    col_wrap=3,
    height=6,
    aspect=19 / 11,
)
ax = g.map_dataframe(
    sns.lineplot,
    x="training_iteration",
    y="Score",
    hue="GNN",
    palette=mpnn_pal,
    estimator="mean",
    errorbar="se",
    legend="full",
    marker=".",
    markeredgecolor=(0, 0, 0, 0.5),
)
g.set_titles(col_template="{col_name}")
g.add_legend(label_order=sorted(d_all_dfs["GNN"].unique()))
sns.move_legend(g, "lower right")

for ax in g.axes.flat:
    ax.set_xlabel("Epochs")  # Set x label
    ax.set_ylabel("Squaerd Error")  # Set y label

# Define new text for each title based on their text
title_new_text = {
    "train_loss": "Train Loss",
    "mean_worse_abs_errs_graph": "Mean of the Absolute Error of the Worst $g \in \Omega$",
    "med_abs_error": "Median Absolute Error",
    "loss": "Validation Loss",
    "max_abs_err_pattern": "Absolute Error of the Worst Predicted Pattern",
}

for ax, title in zip(g.axes.flat, g.col_names):
    ax.set_title(title_new_text.get(title, 'TITLENOTFOUND'),)

plt.tight_layout()
plt.savefig(SAVE_PATH / "average_breakdown_d.pdf", dpi=1200)
plt.close()

In [ ]:
g = sns.FacetGrid(
    nd_all_dfs[~nd_all_dfs["Metric"].str.contains("q")],
    sharex=True,
    sharey=False,
    col="Metric",
    col_wrap=3,
    height=6,
    aspect=19 / 11,
)
ax = g.map_dataframe(
    sns.lineplot,
    x="training_iteration",
    y="Score",
    hue="GNN",
    palette=mpnn_pal,
    estimator="mean",
    errorbar="se",
    legend="full",
    marker=".",
    markeredgecolor=(0, 0, 0, 0.5),
)
g.set_titles(col_template="{col_name}")
g.add_legend(label_order=sorted(d_all_dfs["GNN"].unique()))
sns.move_legend(g, "lower right")

for ax in g.axes.flat:
    ax.set_xlabel("Epochs")  # Set x label
    ax.set_ylabel("Squaerd Error")  # Set y label

# Define new text for each title based on their text
title_new_text = {
    "train_loss": "Train Loss",
    "mean_worse_abs_errs_graph": "Mean of the Absolute Error of the Worst $g \in \Omega$",
    "med_abs_error": "Median Absolute Error",
    "loss": "Validation Loss",
    "max_abs_err_pattern": "Absolute Error of the Worst Predicted Pattern",
}

for ax, title in zip(g.axes.flat, g.col_names):
    ax.set_title(title_new_text.get(title, 'TITLENOTFOUND'),)

plt.tight_layout()
plt.savefig(SAVE_PATH / "average_breakdown_nd.pdf", dpi=1200)
plt.close()

In [ ]:
best_run_nd = ""
best_expr_nd = ""
best_value_nd = np.inf
best_value_idx_nd = -1

best_run_d = ""
best_expr_d = ""
best_value_d = np.inf
best_value_idx_d = -1

best_run_nd_gin = ""
best_expr_nd_gin = ""
best_value_nd_gin = np.inf
best_value_idx_nd_gin = -1

for experiment in sorted(os.listdir(BASE_EXPERIMENT_DIRECTORY / MODELS_RESULTS)):
    trial_cnt = 0
    for trial in os.listdir(BASE_EXPERIMENT_DIRECTORY / MODELS_RESULTS / experiment):
        if "TorchTrainer" not in trial:
            continue

        try:
            df = pd.read_csv(
                BASE_EXPERIMENT_DIRECTORY
                / MODELS_RESULTS
                / experiment
                / trial
                / "progress.csv"
            )
            min_value = df["loss"].min()
            min_value_id = df["loss"].idxmin()

            if (
                experiment in experiment_map_2.keys()
                and "nd_" in experiment_map_2[experiment]
                and min_value < best_value_nd
            ):
                best_value_nd = min_value
                best_value_idx_nd = min_value_id
                best_run_nd = trial
                best_expr_nd = experiment
            elif min_value < best_value_d:
                best_value_d = min_value
                best_value_idx_d = min_value_id
                best_run_d = trial
                best_expr_d = experiment

            if (
                experiment in experiment_map_2.keys()
                and "nd_" in experiment_map_2[experiment]
                and "gin" in experiment_map_2[experiment]
                and min_value < best_value_nd_gin
            ):
                best_value_nd_gin = min_value
                best_value_idx_nd_gin = min_value_id
                best_run_nd_gin = trial
                best_expr_nd_gin = experiment

        except FileNotFoundError:
            # This is not an error! It only means that a trial was inturrepted and resume later
            print(experiment_map_2[experiment])
            print(experiment, trial, " is incomplete")

best_run_pretty = best_run_nd.split("_")[0] + "_" + best_run_nd.split("_")[1]
print("For ND")
print(
    f"Best Run {best_run_pretty}, with {best_value_nd} at epoch {best_value_idx_nd} in {best_expr_nd} experiment ({experiment_map_2[best_expr_nd]})!"
)

best_run_pretty = best_run_nd_gin.split("_")[0] + "_" + best_run_nd_gin.split("_")[1]
print("\nFor ND - GIN")
print(
    f"Best Run {best_run_pretty}, with {best_value_nd_gin} at epoch {best_value_idx_nd_gin} in {best_expr_nd_gin} experiment ({experiment_map_2[best_expr_nd_gin]})!"
)

best_run_pretty = best_run_d.split("_")[0] + "_" + best_run_d.split("_")[1]
print("\nFor D")
print(
    f"Best Run {best_run_pretty}, with {best_value_d} at epoch {best_value_idx_d} in {best_expr_d} experiment ({experiment_map_2[best_expr_d]})!"
)

In [ ]:
g = sns.FacetGrid(
    nd_all_dfs[
        (nd_all_dfs["Metric"].str.contains("q"))
        & (
            (nd_all_dfs["trial_id"] == "995a1ad7")
            | (nd_all_dfs["trial_id"] == "d9c8c887")
        )
    ],
    sharex=True,
    sharey=False,
    col="Metric",
    col_wrap=5,
    height=6,
    aspect=19 / 11,
)
ax = g.map_dataframe(
    sns.lineplot,
    x="training_iteration",
    y="Score",
    estimator="mean",
    hue="GNN",
    palette={"gin": "#56B4E9", "sage": "#FB6467FF"},
    errorbar=None,
    legend="full",
    marker=".",
    markeredgecolor=(0,0,0,0.5),
)
g.add_legend()
sns.move_legend(g, "center right")

for ax in g.axes.flat:
    ax.set_xlabel("Epochs")  # Set x label
    ax.set_ylabel("Squaerd Error")  # Set y label

# plt.tight_layout()
plt.savefig(SAVE_PATH / "best_quartile_breakdown_nd.pdf", dpi=1200)
plt.close()